In [1]:
import os
import csv
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
import re

In [20]:
import pandas as pd

# Load your data
df = pd.read_csv('Knowledge_Fish_with_ids.csv')

# Split your data into two groups based on the label
group1 = df[df['label'] == 1]
group2 = df[df['label'] == 2]

# Downsample the larger group to match the smaller group's size
group1_downsampled = group1.sample(n=len(group2))

# Combine the downsampled group with the other group
balanced_df = pd.concat([group1_downsampled, group2])

# Shuffle the dataframe if needed
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

# Save the new dataframe to a new CSV file
balanced_df.to_csv('balanced_Knowledge_Fish_with_ids.csv', index=False)

In [3]:
def change(image , Traj):
  for i in range(0, len(Traj), 2):
    pair = Traj[i:i+2]
    center_coordinates = (pair[0], pair[1])
    radius = 15
    color = (0, 0, 255)
    thickness = -1
    image = cv2.circle(image, center_coordinates, radius, color, thickness)
  return image

In [7]:
traj_info = pd.read_csv('balanced_Knowledge_Fish.csv')
behaviors = {}

for index, row in traj_info.iterrows():
      trajectory = eval(row['trajectory'])
      label = row['label']
      print(label)
      L = []
      image = np.zeros((1500, 1500, 3), dtype="uint8")
      if(label not in behaviors):
        behaviors[label] = 0
      else:
        behaviors[label] += 1
      ith_video = behaviors[label]
      path = os.path.join('Trajectory_as_images_Knowledge',str(label),str(ith_video))
      print(path)
      os.makedirs(path , exist_ok=True)
      jth_part = 0
      print(trajectory)
      for traj_frame in trajectory:
        new_image = change(image , traj_frame)
        part_path = os.path.join(path,str(jth_part)+'.jpg')
        #cv2.imwrite(part_path,new_image)
        jth_part += 1

1
Trajectory_as_images_Knowledge\1\0
[[163, 163], [159, 159], [159, 159], [152, 152], [138, 138], [133, 133], [123, 123], [117, 117], [115, 115], [113, 113], [112, 112], [111, 111], [111, 111], [111, 111], [111, 111], [111, 111], [113, 113], [114, 114], [115, 115], [114, 114], [114, 114], [113, 113], [112, 112], [111, 111], [111, 111], [110, 110], [110, 110], [108, 108], [107, 107], [106, 106], [105, 105], [104, 104], [105, 105], [105, 105], [102, 102], [100, 100], [98, 98], [97, 97], [96, 96]]
1
Trajectory_as_images_Knowledge\1\1
[[60, 60], [61, 61], [70, 70], [73, 73], [75, 75], [81, 81], [87, 87]]
1
Trajectory_as_images_Knowledge\1\2
[[47, 47], [60, 60], [69, 69], [84, 84], [103, 103], [108, 108]]
1
Trajectory_as_images_Knowledge\1\3
[[169, 169], [174, 174], [174, 174], [168, 168], [170, 170], [173, 173], [177, 177]]
1
Trajectory_as_images_Knowledge\1\4
[[292, 292], [285, 285], [283, 283], [284, 284], [287, 287], [291, 291], [293, 293], [300, 300]]
2
Trajectory_as_images_Knowledge\2

In [2]:
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    return filename

In [ ]:
#old video_maker
import cv2
import os
path_actions = 'Trajectory_as_images_Knowledge'
for action in os.listdir(path_actions):
  action_vids = path_actions + '/' + action
  print(action_vids)
  for vid in os.listdir(action_vids):
      image_folder = action_vids + '/' + vid
      vid_path = os.path.join('Trajectories_as_videos_Knowledge',str(action))
      os.makedirs(vid_path ,  exist_ok=True)
      #video_name = 'Trajectories_as_videos' + '/' + action + '/' + vid + '.avi'
      count = 0
      images = []

      for img in os.listdir(image_folder):
          if img.endswith(".jpg"):
            images.append(img)
          count+=1

      images = sorted(images, key=extract_number)
      print(images)
      frame = cv2.imread(os.path.join(image_folder, images[0]))
      height, width, layers = frame.shape
      video = cv2.VideoWriter(os.path.join(vid_path,vid + '.avi') , cv2.VideoWriter_fourcc(*'DIVX'), 6 , (width,height))
      for image in images:
          video.write(cv2.imread(os.path.join(image_folder, image)))

      cv2.destroyAllWindows()
      video.release()

In [ ]:
import cv2
import os
import numpy as np
from albumentations import Compose, RandomBrightnessContrast, ShiftScaleRotate

# Define the data augmentation pipeline
aug = Compose([
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    RandomBrightnessContrast(p=0.5)
])

def extract_number(f):
    return int(''.join(filter(str.isdigit, f)))

def augment_frame(frame):
    """Apply data augmentation to a frame."""
    frame = aug(image=frame)['image']
    return frame

def duplicate_frames(frames, target_length):
    """
    Duplicates frames in a video until the total length matches target_length
    """
    max_repeat = 10  # The maximum number of times a video can be repeated.
    original_length = len(frames)
    target_length = min(target_length, max_repeat * original_length)

    while len(frames) < target_length:
        frames = frames * 2
    return frames[:target_length]

path_actions = 'Trajectory_as_images_Knowledge'

# Pre-scan all actions to find the video with the max number of frames
max_frames = 0
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        if os.path.isdir(image_folder):
            num_frames = len([img for img in os.listdir(image_folder) if img.endswith(".jpg")])
            max_frames = max(num_frames, max_frames)

# Now process the videos
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        vid_path = os.path.join('Trajectories_as_videos_Knowledge', str(action))
        os.makedirs(vid_path, exist_ok=True)

        images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")], key=extract_number)

        # Load and duplicate frames
        frames = [cv2.imread(os.path.join(image_folder, img)) for img in images]
        frames = duplicate_frames(frames, max_frames)
        frames_aug = [augment_frame(frame) for frame in frames]

        height, width, layers = frames[0].shape
        video = cv2.VideoWriter(os.path.join(vid_path, vid + '.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 5, (width, height))
        video_aug = cv2.VideoWriter(os.path.join(vid_path, vid + '_augmented.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 5, (width, height))
        print(len(frames))
        for frame in frames:
            video.write(frame)

        for frame in frames_aug:
            video_aug.write(frame)

        cv2.destroyAllWindows()
        video.release()
        video_aug.release()


In [13]:
import cv2
import os
import numpy as np
from albumentations import Compose, RandomBrightnessContrast, ShiftScaleRotate
from keras.utils.data_utils import pad_sequences
print('hello')
# Define the data augmentation pipeline
augmentations = [
    Compose([ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=12, p=1)]),
    Compose([ShiftScaleRotate(shift_limit=0.03, scale_limit=0.03, rotate_limit=13, p=1)])
]
def extract_number(f):
    return int(''.join(filter(str.isdigit, f)))

def augment_frame(frame, aug):
    """Apply data augmentation to a frame."""
    frame = aug(image=frame)['image']
    return frame

def duplicate_frames(frames, target_length):
    """
    Duplicates frames in a video until the total length matches target_length
    """
    max_repeat = 10  # The maximum number of times a video can be repeated.
    original_length = len(frames)
    target_length = min(target_length, max_repeat * original_length)

    while len(frames) < target_length:
        frames = frames * 2
    return frames[:target_length]



path_actions = 'Full_Time_Trajectory_for_Final_images'

# Pre-scan all actions to find the video with the max number of frames
max_frames = 0
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        if os.path.isdir(image_folder):
            num_frames = len([img for img in os.listdir(image_folder) if img.endswith(".jpg")])
            max_frames = max(num_frames, max_frames)

# Now process the videos
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        vid_path = os.path.join('Full_Time_Trajectory_for_Final_Videos', str(action))
        os.makedirs(vid_path, exist_ok=True)

        images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")], key=extract_number)

        # Load and duplicate frames
        frames = [cv2.imread(os.path.join(image_folder, img)) for img in images]
        #frames = duplicate_frames(frames, max_frames)
        #frames = pad_frames(frames , max_frames)
        height, width, layers = frames[0].shape
        video = cv2.VideoWriter(os.path.join(vid_path, vid + '.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 14, (width, height))

        for frame in frames:
            video.write(frame)
        # Write augmented frames into new video files
        
        for i, aug in enumerate(augmentations):
            frames_aug = [augment_frame(frame, aug) for frame in frames]
            video_aug = cv2.VideoWriter(os.path.join(vid_path, vid + f'_augmented{i}.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 14, (width, height))

            for frame in frames_aug:
                video_aug.write(frame)
            
            video_aug.release()

        cv2.destroyAllWindows()
        video.release()


hello


In [14]:
import cv2
import os
import numpy as np
from albumentations import Compose, RandomBrightnessContrast, ShiftScaleRotate
from keras.utils.data_utils import pad_sequences

# Define the data augmentation pipeline
augmentations = [
    Compose([ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=12, p=1)]),
    Compose([ShiftScaleRotate(shift_limit=0.03, scale_limit=0.03, rotate_limit=13, p=1)])
]

def extract_number(f):
    return int(''.join(filter(str.isdigit, f)))

def augment_frame(frame, aug):
    """Apply data augmentation to a frame."""
    frame = aug(image=frame)['image']
    return frame

path_actions = 'Full_Time_Trajectory_for_Final_images'

# Pre-scan all actions to find the video with the max number of frames
max_frames = 0
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        if os.path.isdir(image_folder):
            num_frames = len([img for img in os.listdir(image_folder) if img.endswith(".jpg")])
            max_frames = max(num_frames, max_frames)

# Now process the videos
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    action_curr = len(os.listdir(action_vids))
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        vid_path = os.path.join('Full_Time_Trajectory_for_Final_Videos_Copy', str(action))
        os.makedirs(vid_path, exist_ok=True)
        
        images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")], key=extract_number)

        # Load and duplicate frames
        frames = [cv2.imread(os.path.join(image_folder, img)) for img in images]

        height, width, layers = frames[0].shape
        # Write augmented frames into new video files
        print(vid_path)
        print(len(os.listdir(vid_path)))
        if(len(os.listdir(vid_path)) <= 38):
            for i, aug in enumerate(augmentations):
                frames_aug = [augment_frame(frame, aug) for frame in frames]
                video_aug = cv2.VideoWriter(os.path.join(vid_path, vid + f'_augmented{i}_second.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 14, (width, height))
                for frame in frames_aug:
                    video_aug.write(frame)
                
                video_aug.release()

        cv2.destroyAllWindows()


Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Time_Trajectory_for_Final_Videos_Copy\hypoxia_feeding
39
Full_Tim

In [4]:
import cv2
import os
import numpy as np
from albumentations import Compose, RandomBrightnessContrast, ShiftScaleRotate

print('hello')
# Define the data augmentation pipeline
'''
augmentations = [
    Compose([ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=1)]),
    Compose([ShiftScaleRotate(shift_limit=0.01, scale_limit=0.01, rotate_limit=20, p=1)])
]
'''
augmentations = ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=15, p=1)


def extract_number(f):
    return int(''.join(filter(str.isdigit, f)))

def augment_frame(frame, aug):
    """Apply data augmentation to a frame."""
    frame = aug(image=frame)['image']
    return frame

'''
def apply_same_rotation(frames, angle):
    """Applies the same rotation to all frames using OpenCV."""
    # Define the rotation matrix
    (height, width) = frames[0].shape[:2]
    center = (width / 2, height / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    
    augmented_frames = []
    for frame in frames:
        augmented_frame = cv2.warpAffine(frame, rotation_matrix, (width, height))
        augmented_frames.append(augmented_frame)
    return augmented_frames
'''
def apply_same_rotation(frames, angle):
    """Applies the same rotation to all frames."""
    # Create a new augmentation pipeline with a fixed rotation
    aug = ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=(angle, angle), p=1)

    augmented_frames = []
    for frame in frames:
        augmented_frame = aug(image=frame)['image']
        augmented_frames.append(augmented_frame)
    return augmented_frames


def apply_same_shift(frames, dx, dy):
    """Applies the same shift (translation) to all frames."""
    # Define the translation matrix
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    
    shifted_frames = []
    for frame in frames:
        shifted_frame = cv2.warpAffine(frame, M, frame.shape[1::-1])
        shifted_frames.append(shifted_frame)
    return shifted_frames

path_actions = 'Full_Time_Trajectory_for_Final_images'

# Pre-scan all actions to find the video with the max number of frames
max_frames = 0
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        if os.path.isdir(image_folder):
            num_frames = len([img for img in os.listdir(image_folder) if img.endswith(".jpg")])
            max_frames = max(num_frames, max_frames)

# Now process the videos
for action in os.listdir(path_actions):
    action_vids = path_actions + '/' + action
    for vid in os.listdir(action_vids):
        image_folder = action_vids + '/' + vid
        vid_path = os.path.join('Full_Time_Trajectory_for_Final_Videos', str(action))
        os.makedirs(vid_path, exist_ok=True)

        images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")], key=extract_number)

        # Load and duplicate frames
        frames = [cv2.imread(os.path.join(image_folder, img)) for img in images]
        height, width, layers = frames[0].shape
        video = cv2.VideoWriter(os.path.join(vid_path, vid + '.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 14 , (width, height))
    
        for frame in frames:
            video.write(frame)
        '''
        video_aug = cv2.VideoWriter(os.path.join(vid_path, vid + f'_augmented{0}.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 9, (width, height))
        video_aug_shift = cv2.VideoWriter(os.path.join(vid_path, vid + f'_augmented{1}.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 9, (width, height))
        
        rotate_limit = 180
        angle = np.random.uniform(-rotate_limit, rotate_limit)  # define a single random angle for all frames in the video
        frames_aug = apply_same_rotation(frames, angle)
        shift_x = np.random.uniform(-width * 0.2, width * 0.2)
        shift_y = np.random.uniform(-height * 0.2, height * 0.2)
        frames_aug_shifting = apply_same_shift(frames , shift_x , shift_y)
        
        for frame in frames_aug:
            video_aug.write(frame)
        for frame in frames_aug_shifting:
            video_aug_shift.write(frame)
            
        video_aug.release()
        video_aug_shift.release()
        '''
        video.release()


hello


In [ ]:
import numpy as np
import cv2
import pandas as pd
import os

frame_size = 500

def change_centered(image , frame_points, original_points, centered_points , img_size):
    image_size = img_size  # Assumed size of the image
    color = (0, 255, 0)
    thickness = 5
    radius = 20
    thickness_circles = -1
    print(frame_points)
    for i in range(0, len(frame_points), 2):
        point = [frame_points[i], frame_points[i+1]]

        # Add the current point to the list of original points
        original_points.append(point)
        mean_x = np.mean([p[0] for p in original_points])
        mean_y = np.mean([p[1] for p in original_points])
        shift_x = image_size // 2 - mean_x
        shift_y = image_size // 2 - mean_y

        # Shift the current point
        centered_point = [point[0] + shift_x, point[1] + shift_y]
        centered_points.append(centered_point)

        # Draw the centered point on the image
        if len(centered_points) > 1:
            pt1 = (int(centered_points[-2][0]), int(centered_points[-2][1]))
            pt2 = (int(centered_point[0]), int(centered_point[1]))
            image = cv2.line(image, pt1, pt2, color, thickness)
        center_coordinates = (int(centered_point[0]), int(centered_point[1]))
        #image = cv2.circle(image, center_coordinates, radius, color, thickness_circles)

    return image, original_points, centered_points



traj_info = pd.read_csv('balanced_Knowledge_Fish.csv')
behaviors = {}


new_image = np.zeros((frame_size, frame_size, 3), dtype="uint8")
for index, row in traj_info.iterrows():
    trajectory = eval(row['trajectory'])
    label = row['label']
    L = []
    new_image = np.zeros((frame_size, frame_size, 3), dtype="uint8")
    if(label not in behaviors):
      behaviors[label] = 0
    else:
      behaviors[label] += 1
    ith_video = behaviors[label]
    path = os.path.join('Trajectory_as_images_Knowledge_new',str(label),str(ith_video))
    os.makedirs(path , exist_ok=True)
    jth_part = 0

    original_points = []
    centered_points = []
    ct = 0
    for traj_frame in trajectory:
      new_image, original_points, centered_points = change_centered(new_image  , traj_frame , original_points, centered_points, frame_size)
      ct += 1
      part_path = os.path.join(path,str(jth_part)+'.jpg')
      cv2.imwrite(part_path,new_image)
      if(ct % 1 == 0):
         jth_part += 1



In [3]:
import numpy as np
import cv2
import pandas as pd
import os

frame_size = 500


def change_centered(image , frame_points, original_points, centered_points , img_size):
    image_size = img_size  # Assumed size of the image
    color = (0, 255, 0)
    thickness = 5
    radius = 20
    thickness_circles = -1

    print(frame_points)
    for i in range(0, len(frame_points), 2):
        point = [frame_points[i], frame_points[i+1]]

        # Assuming that original_points and centered_points are lists of lists,
        # where each sub-list corresponds to a different object
        if i // 2 >= len(original_points):
            original_points.append([])
            centered_points.append([])

        # Add the current point to the list of original points for the current object
        original_points[i // 2].append(point)
        mean_x = np.mean([p[0] for p in original_points[i // 2]])
        mean_y = np.mean([p[1] for p in original_points[i // 2]])
        shift_x = image_size // 2 - mean_x
        shift_y = image_size // 2 - mean_y

        # Shift the current point
        centered_point = [point[0] + shift_x, point[1] + shift_y]
        centered_points[i // 2].append(centered_point)

        # Draw the centered point on the image as a circle
        center_coordinates = (int(centered_point[0]), int(centered_point[1]))
        image = cv2.circle(image, center_coordinates, radius, color, thickness_circles)

        # If this isn't the first point for the current object, draw a line from the previous point to the current one
        if len(centered_points[i // 2]) > 1:
            prev_point = (int(centered_points[i // 2][-2][0]), int(centered_points[i // 2][-2][1]))
            #image = cv2.line(image, prev_point, center_coordinates, color, thickness)

    return image, original_points, centered_points


traj_info = pd.read_csv('balanced_Knowledge_Fish.csv')
behaviors = {}

new_image = np.zeros((frame_size, frame_size, 3), dtype="uint8")
for index, row in traj_info.iterrows():
    trajectory = eval(row['trajectory'])
    label = row['label']

    new_image = np.zeros((frame_size, frame_size, 3), dtype="uint8")
    if(label not in behaviors):
      behaviors[label] = 0
    else:
      behaviors[label] += 1
    ith_video = behaviors[label]
    path = os.path.join('Trajectory_as_images_Knowledge_new',str(label),str(ith_video))
    os.makedirs(path , exist_ok=True)
    jth_part = 0

    # Initiate original_points and centered_points as list of lists
    original_points = [[] for _ in range(len(trajectory[0]) // 2)]
    centered_points = [[] for _ in range(len(trajectory[0]) // 2)]
    ct = 0
    for traj_frame in trajectory:
      new_image, original_points, centered_points = change_centered(new_image, traj_frame, original_points, centered_points, frame_size)
      ct += 1
      part_path = os.path.join(path,str(jth_part)+'.jpg')
      cv2.imwrite(part_path,new_image)
      if(ct % 1 == 0):
         jth_part += 1


[286, 3]
[268, 1]
[240, 1]
[203, 7]
[187, 12]
[156, 24]
[121, 37]
[118, 43]
[110, 53]
[109, 42]
[104, 37]
[107, 30]
[111, 31]
[112, 29]
[91, 42]
[74, 69]
[73, 80]
[66, 88]
[64, 88]
[65, 86]
[66, 86]
[68, 84]
[68, 85]
[66, 91]
[144, 74]
[149, 76]
[148, 75]
[149, 75]
[150, 79]
[287, 33]
[281, 33]
[273, 37]
[255, 38]
[246, 39]
[252, 37]
[256, 37]
[267, 39]
[267, 48]
[266, 54]
[269, 55]
[278, 58]
[279, 65]
[279, 64]
[279, 68]
[270, 71]
[273, 71]
[289, 84]
[255, 79]
[243, 77]
[230, 68]
[222, 69]
[221, 65]
[217, 66]
[190, 66]
[182, 66]
[175, 77]
[177, 72]
[187, 64]
[199, 53]
[206, 49]
[214, 50]
[218, 56]
[238, 59]
[231, 60]
[223, 63]
[229, 65]
[234, 63]
[243, 61]
[244, 63]
[244, 63]
[245, 60]
[251, 70]
[251, 77]
[262, 84]
[260, 91]
[261, 87]
[250, 92]
[255, 93]
[253, 91]
[254, 91]
[277, 113]
[270, 127]
[240, 170]
[232, 166]
[31, 13]
[31, 15]
[28, 25]
[29, 29]
[29, 35]
[28, 38]
[28, 40]
[27, 41]
[26, 41]
[25, 42]
[24, 45]
[21, 45]
[20, 45]
[20, 44]
[23, 44]
[117, 52]
[122, 45]
[130, 42]
[135,

In [2]:
import numpy as np
import cv2
import pandas as pd
import os
import random
import matplotlib.pyplot as plt

frame_width = 1500
frame_height = 1200

'''
def id_to_color(object_id):
    # Use modulo operation to get a value in range [0, 1]
    hue = object_id % 360 / 360.0

    # Convert HSV color to RGB
    color = plt.cm.hsv(hue)[:3]

    # Convert RGB color from 0-1 range to 0-255 range
    color = tuple(int(c * 255) for c in color)

    return color
'''
# This dictionary will store the color for each object_id
id_to_color_map = {}
frame_size = 1000

def id_to_color(object_id):
    # If the color for the given object_id is already generated,
    # return the color
    if object_id in id_to_color_map:
        return id_to_color_map[object_id]

    # The golden ratio conjugate
    golden_ratio_conjugate = 0.618033988749895

    # Generate a random hue, saturation and value
    hue = ((object_id * golden_ratio_conjugate) % 1) # Change the factor to influence the change rate

    # Convert HSV color to RGB
    color = plt.cm.hsv(hue)[:3]

    # Convert RGB color from 0-1 range to 0-255 range
    color = tuple(int(c * 255) for c in color)

    # Store the generated color in the dictionary
    id_to_color_map[object_id] = color

    return color



def change_centered(image , frame_points, id_s_tracked, original_points, centered_points , img_size):
    image_size = img_size
    color = (0, 255, 0)
    thickness = 13
    radius = 15
    thickness_circles = -1

    # This dictionary will store points according to the id_s_tracked
    point_dict = {}
    for id, i in enumerate(range(0, len(frame_points), 2)):
        point = [frame_points[i], frame_points[i+1]]
        point_dict[id_s_tracked[id]] = point

    for id, point in point_dict.items():

        # Add the current point to the list of original points
        if id not in original_points:
            original_points[id] = []
        original_points[id].append(point)

        mean_x = np.mean([p[0] for p in original_points[id]])
        mean_y = np.mean([p[1] for p in original_points[id]])
        shift_x = image_size // 2 - mean_x
        shift_y = image_size // 2 - mean_y

        # Shift the current point
        centered_point = [point[0] + shift_x, point[1] + shift_y]

        # Add the centered point to the list of centered points
        if id not in centered_points:
            centered_points[id] = []
        centered_points[id].append(centered_point)
        color_generate = id_to_color(id)
        # Draw the centered point on the image
        if len(centered_points[id]) > 1:
            pt1 = (int(centered_points[id][-2][0]), int(centered_points[id][-2][1]))
            pt2 = (int(centered_point[0]), int(centered_point[1]))
            #image = cv2.line(image, pt1, pt2, color_generate, thickness)

        center_coordinates = (int(centered_point[0]), int(centered_point[1]))
        c_cor = (int(point[0]), int(point[1]))
        #image = cv2.circle(image, center_coordinates, radius, color_generate, thickness_circles)
        image = cv2.circle(image, c_cor, radius, color_generate, thickness_circles)

    return image, original_points, centered_points


traj_info = pd.read_csv('Neveen_Videos_Full_Time.csv')
behaviors = {}

for index, row in traj_info.iterrows():
    trajectory = eval(row['trajectory'])
    ids_tracked = eval(row['ids'])
    label = row['label']

    new_image = np.zeros((frame_height, frame_width, 3), dtype="uint8")
    if(label not in behaviors):
      behaviors[label] = 0
    else:
      behaviors[label] += 1
    ith_video = behaviors[label]
    path = os.path.join('Full_Time_Trajectory_for_Final_images',str(label),str(ith_video))
    print(path)
    os.makedirs(path , exist_ok=True)
    jth_part = 0

    # Initiate original_points and centered_points as dictionaries
    original_points = {}
    centered_points = {}
    ct = 0
    for traj_frame, ids_being in zip(trajectory , ids_tracked):
        new_image, original_points, centered_points = change_centered(new_image, traj_frame, ids_being , original_points, centered_points, frame_size)
        ct += 1
        part_path = os.path.join(path,str(jth_part)+'.jpg')
        cv2.imwrite(part_path,new_image)
        jth_part += 1
        if(ct % 59 == 0):
            new_image = np.zeros((frame_size, frame_size, 3), dtype="uint8")


Full_Time_Trajectory_for_Final_images\hypoxia_feeding\0
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\1
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\2
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\3
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\4
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\5
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\6
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\7
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\8
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\9
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\10
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\11
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\12
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\13
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\14
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\15
Full_Time_Trajectory_for_Final_images\hypoxia_feeding\16
Full_Time_Trajectory_for_Final_images\hyp